# DMRG Notes
+ date: 2017-05-26
+ tags: DMRG
+ summary: The density matrix renormalization group (DMRG) is a numerical variational technique devised to obtain the low energy physics of quantum many-body systems with high accuracy. It was invented in 1992 by Steven R. White and it is nowadays the most efficient method for 1-dimensional systems.

<span class="mathmacros" style="display:none;">
$\def\bra#1{\mathinner{\langle{#1}|}}
\def\ket#1{\mathinner{|{#1}\rangle}}
\def\braket#1{\mathinner{\langle{#1}\rangle}}
\newcommand{\mdef}{\overset{\mathrm{def}}{=}}
\newcommand{\bm}{\mathbf}
\newcommand{\pfr}[2]{\frac{\pp #1}{\pp #2}}      % Partial derivative
\newcommand{\pfr}[2]{\frac{\pp #1}{\pp #2}}      % Partial derivative
\newcommand{\inv}[1]{#1^{-1}}   % Inverse Matrix
\newcommand{\invt}[1]{#1^{-T}}  % Inverse Transposed Matrix
\renewcommand{\nl}{\\&\phantom{{}={}}}% Newline In aligned equations
\newcommand{\dfr}[2]{\frac{\dd #1}{\dd #2}}      % Total derivative
\DeclareMathOperator{\Var}{Var}
\DeclareMathOperator{\det}{det}
\DeclareMathOperator{\tr}{tr}
\DeclareMathOperator{\E}{E}
\DeclareMathOperator{\Cov}{Cov}
\DeclareMathOperator{\Beta}{B}
\DeclareMathOperator{\Bdist}{Beta}
\DeclareMathOperator{\sgn}{sgn}
\DeclareMathOperator{\adj}{adj}
\DeclareMathOperator{\ii}{i}
\DeclareMathOperator{\dd}{d}
\newcommand{\pp}{\partial}
\DeclareMathOperator{\rhs}{RHS}
\DeclareMathOperator{\lhs}{LHS}$
</span>

## Basics
+ For site $n$, there is $d$ matrices with dimension $\chi_{n-1}\times\chi_n$. i.e. Tensor with dimension $(d, \chi_{n-1}, \chi_n)$ 
+ For increase of one site, the number of parameters increase by $\chi^2d$
+ __Canonical form__: How to decompose $A\rightarrow \Gamma\Lambda$? Bipartition by schmidt partition, and it will give $\Lambda$
+ **SVD**: For a matrix $M$, find a decomposition $M=\sum_{i=1}^rc_i\ket{\psi_i}\bra{\phi_i}$, where $\psi_i, \phi_i$ are orthonormal, respectively.
+ Suppose we have wave function $\Psi=\sum_{i=1}^{s} c_i\ket{\psi_i}\bra{\phi_i}$, which are unnecessarily orthonormal, the corresponding matrix has rank $r\leq s$.

## MPS
+ At each point, we can decompose a state into left/right part, such that it forms $\Gamma\Lambda\cdots\Gamma$
+ Infinite chain has translational invariance, to confirm the orthogonality:
    + We can use the transfer matrix $T$ to take advantage of invariance
    + The result will be $\propto$ infinite matrix power $T^n$ thus the dominant eigenvalue must be 1
    + For more complex case, we can generalize the $T$ matrix by a multiple contraction
    + Local operator will affect only one site, so it is plugged in the contraction between legs of $\Gamma$

## iTEBD
Like power method, apply a function monotonic increase function $f_t$ of the Hamiltonian, which will become extremely sharp as parameter $t$ goes to infinity, s.t. 
$$\lim_{t\to\infty} f_t(\hat H)\ket{\psi}=\ket{0}$$
+ For Power method, it is $f_t(x)=|x|^t$, so it can find the eigen vector with largest absolute eigenvalue
+ For iTEBD, it is $f_t(x)=e^{-tx}$, so it can find eigenvector of lowest eigenvalue

### Implementation
To apply iTEBD, we need to expand $f_t(\hat H)$ by Trotter-Suzuki expansion:
$$e^{(A+B)\delta}\approx e^{A\delta}e^{B\delta}+O(\delta^2)$$
+ $A, B$ is the local Hamiltonian operator of odd/even sites $H_{\rm{odd}}, H_{\rm{even}}$, respectively
+ The elements in expanded operator should be commutable, so that it is readily computable
+ For more complex interaction than nearest interaction, there should be more terms like $C, D, E\ldots$
+ The one site translational invariance has broken as the $A, B$ are for two sites 

## MPO
Expressing the Hamiltonian of the system in terms of matrix product operator (MPO). 

For matrix $M=ABC$,
$$M_{il}=A_{ij}B_{jk}C_{kl}, \quad i\geq j\geq k\geq l$$ only diagonal elements can be used multiple times. If we care about $M_{n1}$ only, then we are finding a path from $n$ to $1$ passing $i,k,l\ldots$
+ One site interaction $n\to 1$ will pass element $(n, 1)$
+ Two sites interaction $n\to i\to 1$ will pass elements $(n, i)$ and $(i, 1)$
    + Multiple two site interaction can be completed by $L$ shaped matrices with $1$ at ends, which has $n-2$ kinds of paths
+ Convoluted multi-interaction? Not physical? $k$ sites $n\to i_{k-1}\cdots\to i_1\to 1$
    + 3 sites, indepedent interaction, $\dfrac{n-2}{2}$ kinds?
+ Sub-nearest interaction? Set some matrices at Convoluted multi-interaction to be $1$

For $2\times 2$ matrix
$$M=\begin{bmatrix}C_i& 0\\B_i & A_i \end{bmatrix}$$
the only path survives is $A\to B\to C$ where $A, C$ can be repeated for any times as they are diagonal

In [477]:
from MPO import MPO as O
from importlib import reload
reload(MPO);

### Three sites interaction

$$\sum_i \left[A_{i-1}\otimes B_{i}\otimes C_{i+1}+D_{i-1}\otimes E_{i}\otimes F_{i+1}\right]$$

In [480]:
O('D')*O('E')*O('F')+O('A')*O('B')*O('C')

 1                      
 F   ↰                  
     E   ↰              
 C           ↰          
             B   ↰      
         D       A   1  

There are two paths: $6\to5\to4\to1$ and $6\to3\to2\to1$, so

If $B=E=1$, then this MPO element decribes a sub-nearest interaction. 

### Simple subnearest interaction
Consider Hamiltonian $H=H_0+H_1+H_2$ where $H_0=\sum A$ is one site, $H_1=\sum B_i\otimes C_{i+1}$ for nearest product, and $H_2=\sum D_{i}\otimes E_{i+2}$ for subnearest product, i.e.
$$\sum A_i+\sum B_{i}\otimes C_{i+1}+\sum D_{i-1}\otimes 1_i\otimes F_{i+1}$$

The MPO is

In [481]:
O('A')+O('B')*O('C')+O('D')*O('1')*O('E')

 1                  
 C   ↰              
 E       ↰          
         1   ↰      
 A   B       D   1  

### Picture on MPO
Diagramatically, the path should go from bottom line to left line by the way:
+ Starting from bottom line with $\uparrow$ direction
+ When going in $\uparrow$ direction, reflected by diagonal element(may stay for some turns) and then go to the left direction
+ When going in $\leftarrow$ direction, scattered by an element
+ Ending at left line with $\uparrow$ direction

## iDMRG
Advantages:
+ Does not rely on Trotter-Suzuki decomposition, so it can be applied to longer range interactions
    + Why not iTEBD? Is there a more complicated T-S series? 
+ High efficiency especially for large correlation length and small energy gap

For left/right block, the Hilbert space is $\chi$, while for each new sites is $d$. The total Hilbert space has dimension $\chi^2d^2$ thus the Hamiltonian is $\chi^4d^4$. After computation, truncation is needed to retain a low complexity.
+ We have the bases of $L/R$ environment $\alpha\beta$ and new sites $jk$, i.e. $\ket{\alpha j\beta k}$
+ With MPO, the Hamiltonian can be expressed on such bases?
+ Apply SVD decomposition to reduced density matrix of ground state and get approximation for new environment 
+ New $L/R$ blocks

## Finite DMRG
The difference to iDMRG is additional back-and-forth sweep. During the sweep, one block increase and the other decrease until reaching the end. 

## References
+ Infinite size density matrix renormalization group, revisited by *I.P. McCulloch*
+ The phase diagram of the anisotropic spin-2 XXZ model: an infinite system DMRG study by *Jonas A. Kjäll, Michael P. Zaletel, Roger S. K. Mong et.al*